# Wishart Process for Metereological Data

In [ ]:
import numpy as np
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
from datetime import datetime

from regain.bayesian import wishart_process_

In [ ]:
# load data
df = pd.read_csv("../../regain/data/metereological.csv").replace("M", np.NaN) # .fillna(0)

In [ ]:
df["temp"] = (df["MAX AT C"].values.astype(float) + df["MIN AT C"].values.astype(float)) / 2.

df['year_month'] = df.DATE.str.split("/").apply(lambda x: "/".join(x[:-1]))

df_temp = df[df.temp.notna()].fillna(0).copy()
df_temp['SNOW cm'] = df_temp['SNOW cm'].values.astype(float)
df_temp['RAIN cm'] = df_temp['RAIN cm'].values.astype(float)
df_temp["precipitation"] = df_temp['SNOW cm'] + df_temp["RAIN cm"]

idx_ok = [k for k, v in df_temp.groupby("year_month").groups.items() if len(v) > 2]

df_filt = df_temp[df_temp.year_month.isin(idx_ok)]

X = df_filt[["temp", "precipitation"]]
y = df_filt.year_month
y = [i.days for i in pd.to_timedelta(pd.to_datetime(y))]

xaxis = pd.to_datetime(df_filt.year_month)

wp = wishart_process_.WishartProcess(n_iter=500, verbose=True, learn_ell=False)
wp.fit(X, y)

In [ ]:
py.init_notebook_mode(connected=True)
py.iplot([go.Scatter(y=wp.covariance_[:, 0,1], x=xaxis, mode='markers+lines')])